<a href="https://colab.research.google.com/github/SerSanC/Master-Degree-Artificial-Inteligent/blob/main/ProjectSupervisedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Preprocessing
---------

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
import torchvision
from torch.utils.data import DataLoader
from google.colab import drive
import random
import shutil
import torch
from torchvision import datasets, transforms
import helper

In [ ]:
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

path = '/content/drive/MyDrive/Colab Notebooks/Datos/Sergi/eyes'

transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])


dataset = datasets.ImageFolder(path,transform=transform)
classes = dataset.classes
data_loader = DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5 
    plt.imshow(np.transpose(img, (1, 2, 0)))

dataiter = iter(data_loader)
images, labels = dataiter.next()

fig = plt.figure(figsize=(20, 4))

for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [ ]:
from torch.utils.data import random_split

train_set, test_set = random_split(dataset, (int(len(dataset) * 0.7) + 1, int(len(dataset) * 0.3)))
train_set, valid_set = random_split(train_set, (int(len(train_set) * 0.7) + 1, int(len(train_set) * 0.3)))

train_set

In [ ]:
train_loader = DataLoader(train_set, batch_size=64)
valid_loader = DataLoader(valid_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

# 2.- CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(46656, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
from torch.optim import Adam
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn = Net().to(device)

criterion = nn.CrossEntropyLoss()
# params = cnn.resnet.fc.parameters()
params = cnn.parameters()
optimizer = Adam(params, lr=0.003)

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, device):
  total_step = len(train_loader)
  num_epochs = 10
  train_losses = []
  valid_losses = []
  for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    for i, (img, target) in enumerate(train_loader):
      img = img.to(device)
      target = target.to(device)

      optimizer.zero_grad()

      output = model(img)
      

      loss = criterion(output, target)

      loss.backward()
      optimizer.step()

      train_loss += loss.item() * img.size(0)

    model.eval()
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)
      valid_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))

  print(f"Training Losses: {train_losses}")  
  print(f"Valid Losses: {valid_losses}")  
  plt.plot(train_losses, label='Training loss')
  plt.plot(valid_losses, label='Validation loss')
  plt.legend()
  plt.show()

In [ ]:
train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)

In [ ]:
def global_accuracy(model, test_loader):
  correct = 0
  total = 0
  model.to("cpu")
  dataiter = iter(test_loader)
  with torch.no_grad():
    for data in dataiter:
      img, label = data
      output = model(img)
      _, predicted = torch.max(output.data, 1)
      total += label.size(0)
      correct += (predicted == label).sum().item()

  print(f"Accuracy: {100 * correct / total}")

def accuracy_per_class(model, test_loader, classes, device):
  class_correct = list(0. for i in range(5))
  class_total = list(0. for i in range(5))
  cnn.to(device)
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.to(device)
          labels = labels.to(device)
          outputs = cnn(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          if(c.item()):
            class_correct[labels.item()] += 1
          class_total[labels.item()] += 1
  

  accuracy = 0
  for i in range(0,2):
    print
    if int(class_total[i]) == 0:
      accuracy = 0
    else:
      accuracy = float(class_correct[i] / class_total[i])
    print(f"{classes[i]} | Correct: {class_correct[i]} | Total: {class_total[i]}" +
          f" | Accuracy: {accuracy}")



In [ ]:
global_accuracy(cnn, test_loader)

In [ ]:
accuracy_per_class(cnn, test_loader, classes, device)

In [ ]:
torch.save(cnn.state_dict(), "cnn_basic.pt")

In [ ]:
model = Net()
model.load_state_dict(torch.load('cnn_basic.pt'))

In [ ]:
from torchvision import models,transforms

class ClassifierResnet34(nn.Module):
  def __init__(self):
    super(ClassifierResnet34, self).__init__()
    self.resnet = models.resnet34(pretrained=True)
    self.resnet.fc = nn.Linear(self.resnet.fc.in_features,5)
    

  def forward(self, image):
    output = self.resnet(image)
    return output

In [ ]:
from torch.optim import Adam

cnn = ClassifierResnet34().to(device)

criterion = nn.CrossEntropyLoss()
params = cnn.resnet.fc.parameters()
optimizer = Adam(params, lr=0.003)

In [ ]:
train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)

In [ ]:
global_accuracy(cnn, test_loader)
accuracy_per_class(cnn, test_loader, classes, device)

In [ ]:
from torchvision import models,transforms

class ClassifierResnet50(nn.Module):
  def __init__(self):
    super(ClassifierResnet50, self).__init__()
    self.resnet = models.resnet50(pretrained=True)
    self.resnet.fc = nn.Linear(self.resnet.fc.in_features,5)
    

  def forward(self, image):
    output = self.resnet(image)
    return output

In [ ]:
from torch.optim import Adam

cnn = ClassifierResnet50().to(device)

criterion = nn.CrossEntropyLoss()
params = cnn.resnet.fc.parameters()
optimizer = Adam(params, lr=0.003)

In [ ]:
train_model(cnn, train_loader, valid_loader, criterion, optimizer, device)

In [ ]:
global_accuracy(cnn, test_loader)
accuracy_per_class(cnn, test_loader, classes, device)

--------
# Experimentos

In [ ]:
from torchvision import models,transforms

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(25088, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = self.fc3(x)
        return x


In [ ]:
from torch.optim import Adam,SGD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #training with either cpu or cuda

model = VGG16() #to compile the model
model = model.to(device=device) #to send the model for training on either cuda or cpu

## Loss and optimizer
learning_rate = 1e-4 #I picked this because it seems to be the most used by experts
load_model = True
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr= learning_rate) #Adam seems to be the most popular for deep learning

In [ ]:
train_model(model, train_loader, valid_loader, criterion, optimizer, device)

In [ ]:
global_accuracy(model, test_loader)
accuracy_per_class(model, test_loader, classes, device)

In [ ]:
from torch.optim import Adam,SGD

## Load the model based on VGG19
vgg_based = torchvision.models.vgg19(pretrained=True)

## freeze the layers
for param in vgg_based.parameters():
   param.requires_grad = False

# Modify the last layer
number_features = vgg_based.classifier[6].in_features
features = list(vgg_based.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(number_features, len(classes))])
vgg_based.classifier = torch.nn.Sequential(*features)

vgg_based = vgg_based.to(device)

print(vgg_based)

criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = SGD(vgg_based.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_model(vgg_based, train_loader, valid_loader, criterion, optimizer_ft, device)

In [ ]:
global_accuracy(vgg_based, test_loader)
accuracy_per_class(vgg_based, test_loader, classes, device)